In [2]:
!pip install pydantic


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install langchain_openai

zsh:1: /Users/prajwol/Documents/travories-geolocation/LangGraph/venv/bin/pip: bad interpreter: /Users/prajwol/Documents/travories-geolocation/LangGraph/GraphVisulaization/venv/bin/python3.12: no such file or directory
  Using cached langchain_openai-0.3.14-py3-none-any.whl.metadata (2.3 kB)
  Using cached langchain_core-0.3.56-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.3.39-py3-none-any.whl.metadata (15 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (3.0 kB)
Using cached langchain_openai-0.3.14-py3-none-any.whl (62 kB)
Using cached langchain_core-0.3.56-py3-none-any.whl (437 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
Using cached langsmith-

In [4]:
!pip install langchain_google_genai

zsh:1: /Users/prajwol/Documents/travories-geolocation/LangGraph/venv/bin/pip: bad interpreter: /Users/prajwol/Documents/travories-geolocation/LangGraph/GraphVisulaization/venv/bin/python3.12: no such file or directory
  Using cached langchain_google_genai-2.1.4-py3-none-any.whl.metadata (5.2 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
  Using cached google_api_core-2.24.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached grpcio_status-1.72.0rc1-py3-none-any.whl.metadata (1.1 kB)
  Using cached protobuf-6.31.0rc2-cp39-abi3-macosx_10_9_universal2.whl.metadata (596 bytes)
  Using cached grpcio-1.72.0rc1-cp312-cp312-macosx_11_0_universal2.whl.metadata (3.8 kB)
Using cached langchain_google_genai-2.1.4-py3-none-any.whl (44 kB)
Using cached filetype-1.2.0-py2.py3-none-any.whl (19 kB)
Using cached google_ai_gener

In [5]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
# 1. Using Pydantic BaseModel

class NepalPackageTour(BaseModel):

    """Information about a Nepal Package Tour"""

    name: str = Field(description="Title of the Package")
    description: str = Field(description="Description of the Package")
    price: float = Field(description="Price of the Package")
    duration: int = Field(description="Duration of the Package")
    location: str = Field(description="Location of the Package")
    features: list[str] = Field(description="Features of the Package")

structured_llm_Package = llm.with_structured_output(NepalPackageTour)
result_Package = structured_llm_Package.invoke("Give Me a Package to visit in nepal in this summer")
print("Package Info:\n", result_Package)

Package Info:
 name='Nepal Summer Package' description='This package includes all the major tourist destinations in Nepal.' price=1000.0 duration=7 location='Nepal' features=['Kathmandu sightseeing', 'Pokhara sightseeing', 'Chitwan National Park safari']


In [7]:
# 2. Using TypedDict with Annotated

from typing_extensions import Annotated, TypedDict
from typing import Optional

class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    # rating: Annotated[int, ..., "How funny the joke is, from 1 to 10"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

{'setup': 'Why are cats such bad dancers?',
 'punchline': 'Why are cats such bad dancers? Because they have two left feet!'}

In [8]:
json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline", "rating"],
}

structured_llm_joke_json = llm.with_structured_output(json_schema)
result_joke_json = structured_llm_joke_json.invoke("Tell me a joke about cats")
print("\nJoke (JSON Schema):\n", result_joke_json)

Key 'parameters' is not supported in schema, ignoring



Joke (JSON Schema):
 {'rating': 7.0, 'setup': 'Why are cats such bad dancers?', 'punchline': 'Why are cats such bad dancers? Because they have two left feet!'}
